# 目的
TensorflowやTheanoのラッパーである`Keras`を使う練習。

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import os
import pickle
import pandas as pd
import numpy as np

from FX.FX import SQLAnaforFX
from FX.FX import drawfigfunc as dff
from FX.FX import analyzefuncs as af
# from FX.FX import downloadFXdata as dFX

## データの用意

In [5]:
"""DBの読み込み"""
dbpath = "201704"
sql = SQLAnaforFX(dbpath)
sql.showtablenames()

"""OHLC"""
close = sql.toDataFrame("ask01min", colselect=["close"]).as_matrix()[:,0]
opens = sql.toDataFrame("ask01min", colselect=["open"]).as_matrix()[:,0]
high = sql.toDataFrame("ask01min", colselect=["high"]).as_matrix()[:,0]
low = sql.toDataFrame("ask01min", colselect=["low"]).as_matrix()[:,0]
ohlc = np.vstack((opens, high, low, close))

dclose = np.zeros_like(close)
dclose[1:] = np.diff(close)

# Calculate mean spread
close_bid = sql.toDataFrame("bid01min", colselect=["close"]).as_matrix()
s = np.mean((close - close_bid)[close != 0])

# BMIの学習
やはりBMIのデータを用いて学習させてみる。次はIrisでもテストするか？

## データの正規化と分離

In [18]:
# BMIのデータを読み込んで正規化する --- (※1)
csv = pd.read_csv("./data/bmi.csv")
# 体重と身長のデータ
csv["weight"] /= 100
csv["height"] /= 200
X = csv[["weight", "height"]].as_matrix() # --- (*1a)

# ラベル
bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
y = np.empty((20000,3))
for i, v in enumerate(csv["label"]):
    y[i] = bclass[v]

# 訓練データとテストデータを分ける --- (※2)
train_size = 10000
X_train, y_train = X[1:train_size + 1], y[1:train_size + 1]
X_test,  y_test  = X[train_size + 1:], y[train_size + 1:] 

## モデルの構造の定義
モデルは`Sequential`を用いて、直線的に各レイヤーの定義を追加していく。

In [27]:
# モデルの構造を定義 --- (※3)
model = Sequential() # 初期化

model.add(Dense(128, input_shape=(2,))) # 中間層の第一層。入力層？入力は"height"と"weight"の二つ
model.add(Activation('relu')) # ReLU関数を活性化関数とする
model.add(Dropout(0.1)) # 10%のノードをドロップアウト

model.add(Dense(128)) # 第二層
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(3)) # 第三層は出力層
model.add(Activation('softmax')) # 最後はsoftmax関数を用いてみる

## モデルのコンパイル
この段階で損失関数などを定義する。

In [28]:
# モデルを構築 --- (※4)
model.compile(
    loss='categorical_crossentropy',
    optimizer="rmsprop",
    metrics=['accuracy'])

## 訓練
verbose = 1 にして出力すると、Notebookへの出力の関係で止まる。なのでverbose = 0にすること。

In [35]:
# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=10,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

## 評価

In [36]:
# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 0.0458675569633
accuracy= 0.98039803981


## 所感
Tensorflowを直に利用するよりははるかに直感的な操作が可能であると感じられた。

# Irisのデータ
sklearnのデータセットに入っているirisのデータを用いて学習の練習をしてみる。

In [50]:
iris = load_iris()

## 正解ラベルの加工

In [44]:
def labeling(target):
    uniques = np.sort(np.unique(target))
    size = len(uniques)
    labels = np.zeros((len(target), size), dtype=int)
    for ii in range(size):
        labels[target == uniques[ii], ii] = 1
    
    return labels.copy()
label = labeling(iris.target)

## データの正規化と分離

In [77]:
X = iris.data.copy()
X = X / np.tile((X.max(axis=0))[None, :], (len(X), 1))
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5555)

## モデルの構造の定義

In [88]:
model = Sequential()

# 入力層
model.add(Dense(512, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.1))

# 中間層
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))

# 出力層
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

# コンパイル
model.compile(loss='categorical_crossentropy',
    optimizer="rmsprop",
    metrics=['accuracy'])

## 訓練と評価

In [89]:
# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

In [90]:
# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 0.367143461439
accuracy= 0.95555555688


## モデルの保存
[こちら](http://m0t0k1ch1st0ry.com/blog/2016/07/17/keras/)を参考にする

In [80]:
model_json_str = model.to_json()
with open('./data/iris_mlp_model.json', 'w') as ff:
    ff.write(model_json_str)
model.save_weights('./data/iris_mlp_weights.h5')

### 読み込み

In [91]:
def save_learned_model(model, fldrpath):
    model_json_str = model.to_json()
    f_json = fldrpath + fldrpath.split("/")[]os.path.splitext(fpath)[0] + ".json"
    with open(f_json, 'w') as ff:
        ff.write(model_json_str)
    
    f_h5 = os.path.splitext(fpath)[0] + ".h5"
    model.save_weights(f_h5)

def load_learned_model(fpath):
    """
    
    """
    # モデルの読み込み
    f_json = os.path.splitext(fpath)[0] + ".json"
    model = model_from_json(open(f_json, "r").read())
    
    # 学習結果の読み込み
    f_h5 = os.path.splitext(fpath)[0] + ".h5"
    model.load_weights(f_h5)
    
    # モードの読み込み
    f_mode = os.path.splitext(fpath)[0] + ".mode"
    with open(f_mode, "rb") as ff:
        mode = pickle.load(ff)
        
    # summary = model2.summary()
    model.compile(loss=mode["loss"],
        optimizer=mode["optimizer"],
        metrics=mode["metrics"])
    
    return model

# テスト
score = model2.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 0.186865983407
accuracy= 0.977777777778


## 所感
テストデータが少なくても、ノードの数が多いほうが正解率が上がる？   
ただのフィッティングの感覚でいくと、サンプル数よりも多いパラメータを求めようとしているような気がしてならないのだが、、   
学習済みモデルの保存と読み込みの手順が、Tensorflowより多い。この辺はさらにラッパーを用意すればよいか。

# 為替のデータを学習

## モデルの定義

In [3]:
def create_model(X, y):
    model = Sequential()
    
    # 入力層
    model.add(Dense(512, input_shape=(X.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    # 中間層
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    # 出力層
    model.add(Dense(y.shape[1]))
    model.add(Activation('softmax'))

    # コンパイル
    model.compile(loss='categorical_crossentropy',
        optimizer="rmsprop",
        metrics=['accuracy'])
    
    return model

In [25]:
y = af.labeling(dclose, s, 60, 2)

In [14]:
close_norm = close - close.mean()
X = close_norm / np.abs(close_norm).max()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5555)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2632,) (1128,) (2632, 3) (1128, 3)


## モデルの生成

In [12]:
model = create_model(X[:, None], y)

## 訓練と評価

In [13]:
# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

In [15]:
# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 1.09173293739
accuracy= 0.394503546099


closeだけではやはり40%を超えない。

## OHLCを利用

In [30]:
y = af.labeling(dclose, s, 30, 2)

In [31]:
ohlc_mean = ohlc.mean(axis=1)
ohlc_norm = np.zeros_like(ohlc)
for ii in range(ohlc.shape[0]):
    ohlc_norm[ii] = ohlc[ii] - ohlc_mean[ii]
    ohlc_norm[ii] /= np.abs(ohlc_norm[ii]).max()

X = ohlc_norm.T
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5555)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2632, 4) (1128, 4) (2632, 3) (1128, 3)


In [35]:
model = create_model(X, y)

# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=30,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

In [36]:
# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 1.0883773337
accuracy= 0.38829787234


## dcloseを利用

In [41]:
X = dclose / np.abs(dclose).max()
y = af.labeling(close, s, 60, 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

model = create_model(X[:,None], y)

# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

(3008,) (752,) (3008, 3) (752, 3)
loss= 0.94668527106
accuracy= 0.457446808511


学習用のデータの数が少ない？

## 層を増やしてみる

In [46]:
def create_model(X, y):
    model = Sequential()
    
    # 入力層
    model.add(Dense(512, input_shape=(X.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    # 中間層
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    # 中間層２
    model.add(Dense(512))
    model.add(Activation('tanh'))
    model.add(Dropout(0.1))

    # 出力層
    model.add(Dense(y.shape[1]))
    model.add(Activation('softmax'))

    # コンパイル
    model.compile(loss='categorical_crossentropy',
        optimizer="rmsprop",
        metrics=['accuracy'])
    
    return model

In [47]:
X = dclose / np.abs(dclose).max()
y = af.labeling(close, s, 60, 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

model = create_model(X[:,None], y)

# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

(3008,) (752,) (3008, 3) (752, 3)
loss= 0.947001878251
accuracy= 0.456117021277


時間やほかの情報もパラメータに入れるとよいのだろうが、、

## ラベリングの対象を変える
これまではなぜかdcloseに対してラベリングを行っていた。本当になぜだろう？   
ここでcloseに対してラベリングを行い、予測確率が上がるかどうか確認する。

In [6]:
def create_model(X, y):
    model = Sequential()
    
    # 入力層
    model.add(Dense(512, input_shape=(X.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    # 中間層
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    # 出力層
    model.add(Dense(y.shape[1]))
    model.add(Activation('softmax'))

    # コンパイル
    model.compile(loss='categorical_crossentropy',
        optimizer="rmsprop",
        metrics=['accuracy'])
    
    return model

In [7]:
y = af.labeling(close, s, 60, 2)

close_norm = close - close.mean()
X = close_norm / np.abs(close_norm).max()
X_train, X_test, y_train, y_test = train_test_split(X[:,None], y, test_size=0.3, random_state=5555)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2632, 1) (1128, 1) (2632, 3) (1128, 3)


In [8]:
model = create_model(X[:,None], y)

# データで訓練 --- (※5)
hist = model.fit(
    X_train, y_train,
    batch_size=100,
    epochs=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=0)

# テストデータを用いて評価する --- (※6)
score = model.evaluate(X_test, y_test, verbose=0)
print('loss=', score[0])
print('accuracy=', score[1])

loss= 0.961993373032
accuracy= 0.512411347518


ようやく51％まで上がったが、これが普通なのかもしれない。

# LSTMの練習
Kerasの`LSTM`クラスを用いて、為替の時系列データを学習させてみる。

In [9]:
def create_model(X, y):
    model = Sequential()
#     model.add(Embedding(max_features, 256, input_length=maxlen))
    model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), activation='sigmoid', recurrent_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(y.shape[1]))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

## データの用意

In [10]:
y = af.labeling(close, s, 10, 2)

In [12]:
close_norm = close - close.mean()
X = close_norm / np.abs(close_norm).max()
X = af.timeseries(X,20)
X_train, X_test, y_train, y_test = train_test_split(X[:, :, None], y, test_size=0.3, random_state=5555)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2632, 20, 1) (1128, 20, 1) (2632, 3) (1128, 3)


In [13]:
model = create_model(X[:, :, None], y)
model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=0)
score = model.evaluate(X_test, y_test, batch_size=16, verbose=0)

In [14]:
print('loss=', score[0])
print('accuracy=', score[1])

loss= 1.04494662488
accuracy= 0.419326241135


LSTMの方は、まだまだチューニングが必要そうである。

# まとめ
Kerasを用いると、Tensorflowの利用が楽になることが分かった。   
また、チューニングと適切な特徴量の選択をしない限り、為替の予測はできないことも分かった。